FEDRL: Running 3 agents for HalfCheetah env for 150 training iterations in parallel following PPO algo

In [1]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air, tune

from ray.rllib.env.multi_agent_env import make_multi_agent

from ray.rllib.env.env_context import EnvContext

import gym

MultiHalfCheetah = make_multi_agent("HalfCheetah-v3")


# different ctrl_cost_weight and reset_noise_scale for 3 agents
paramlist = [
    [0.1, 0.1],
    [0.5, 0.3],
    [0.6, 0.8]
]

class WrappedMultiHalfCheetah(MultiHalfCheetah):
    def __init__(self, config: EnvContext = None):
        if config is None:
            config = {}
        num = config.pop("num_agents", 1)

        if isinstance("HalfCheetah-v3", str):
            self.agents = [gym.make("HalfCheetah-v3", ctrl_cost_weight=paramlist[i][0], reset_noise_scale=paramlist[i][1]) for i in range(num)]
        self.dones = set()
        self.observation_space = self.agents[0].observation_space
        self.action_space = self.agents[0].action_space
        self._agent_ids = set(range(num))

/home/rakesh/miniconda3/envs/rllib/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [2]:
env = MultiHalfCheetah()

# Running 3 agents for HalfCheetah env for 150 training iterations in parallel
config = PPOConfig()\
    .environment(WrappedMultiHalfCheetah, env_config={"num_agents": 3})\
    .rollouts(num_rollout_workers=1)\
    .multi_agent(
        policies={
            "policy_0": (
                None, env.observation_space, env.action_space, {"gamma": 0.80}
            ),
            "policy_1": (
                None, env.observation_space, env.action_space, {"gamma": 0.95}
            ),
            "policy_2": (
                None, env.observation_space, env.action_space, {"gamma": 0.92}
            )
        },
        policy_mapping_fn = lambda agent_id: f"policy_{agent_id}",
    )

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [3]:
stopping_criteria = {
    "training_iteration": 150, 
}
tuner = tune.Tuner(
        "PPO",
        tune_config=tune.TuneConfig(
            metric="episode_reward_mean",
            mode="max",
        ),
        param_space=config,
        run_config=air.RunConfig(local_dir='./results_fedRL' ,name="FEDRL_HALFCHEETAH_PPO" ,stop=stopping_criteria, verbose=1),
    )
results = tuner.fit()

2023-05-10 21:53:41,951	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-05-10 21:53:41,952	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-05-10 21:53:41,952	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(pid=132364) /home/rakesh/miniconda3/envs/rllib/lib/python3.10/site-packages/tenso

Running 3 agents for Ant env for 100 training iterations in parallel folling PPO algo

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import air, tune
from ray.tune.logger import pretty_print
from ray.rllib.algorithms.algorithm import Algorithm

from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.policy.policy import PolicySpec

from ray.rllib.env.env_context import EnvContext

import os, random, gym

MultiAnt = make_multi_agent("Ant-v3")


# different ctrl_cost_weight, reset_noise_scale and contact_cost_weight for 3 agents
paramlist = [
    [0.1, 0.1, 0.2],
    [0.5, 0.3, 0.4],
    [0.6, 0.8, 0.5]
]

class WrappedMultiAnt(MultiAnt):
    def __init__(self, config: EnvContext = None):
        if config is None:
            config = {}
        num = config.pop("num_agents", 1)

        if isinstance("Ant-v3", str):
            self.agents = [gym.make("Ant-v3", ctrl_cost_weight=paramlist[i][0], reset_noise_scale=paramlist[i][1], contact_cost_weight=paramlist[i][2]) for i in range(num)]
        self.dones = set()
        self.observation_space = self.agents[0].observation_space
        self.action_space = self.agents[0].action_space
        self._agent_ids = set(range(num))

In [5]:
env = MultiAnt()

# Running 3 agents for Ant env for 100 training iterations in parallel
config = PPOConfig()\
    .environment(WrappedMultiAnt, env_config={"num_agents": 3})\
    .rollouts(num_rollout_workers=1)\
    .multi_agent(
        policies={
            "policy_0": (
                None, env.observation_space, env.action_space, {"gamma": 0.80}
            ),
            "policy_1": (
                None, env.observation_space, env.action_space, {"gamma": 0.95}
            ),
            "policy_2": (
                None, env.observation_space, env.action_space, {"gamma": 0.92}
            )
        },
        policy_mapping_fn = lambda agent_id: f"policy_{agent_id}",
    )

In [6]:
stopping_criteria = {
    "training_iteration": 100, 
}
tuner = tune.Tuner(
        "PPO",
        tune_config=tune.TuneConfig(
            metric="episode_reward_mean",
            mode="max",
        ),
        param_space=config,
        run_config=air.RunConfig(local_dir='./results_fedRL' ,name="FEDRL_ANT_PPO" ,stop=stopping_criteria, verbose=1),
    )
results = tuner.fit()

2023-05-10 22:45:19,907	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-05-10 22:45:19,909	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-05-10 22:45:19,909	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(pid=135479) /home/rakesh/miniconda3/envs/rllib/lib/python3.10/site-packages/tenso

It is possible to transform privacy preserving Federated setting where the meta policy is learned at the global level without sharing the samples generated by each individual agent.
MBMPO closely works like federated leaning. It uses an ensembles of transition dynamics models for learning. Its task is to learn a policy that can perform well across the ensemble. So we could have several parallel agents in a federated setting and collectively build a global model without sharing.